In [1]:
from lattedb.quarksmear.models import GaugeCovariantGaussian as quarksmear_GaugeCovariantGaussian
from lattedb.fermionaction.models import Hisq as fermionaction_Hisq
from lattedb.gaugeaction.models import LuescherWeisz as gaugeaction_LuescherWeisz
from lattedb.fermionaction.models import MobiusDW as fermionaction_MobiusDW
from lattedb.gaugeconfig.models import Nf211 as gaugeconfig_Nf211
from lattedb.quarksmear.models import Point as quarksmear_Point
from lattedb.linksmear.models import Unsmeared as linksmear_Unsmeared
from lattedb.linksmear.models import WilsonFlow as linksmear_WilsonFlow
from lattedb.propagator.models import OneToAll as propagator_OneToAll

In [5]:
gaugeconfigs = gaugeconfig_Nf211.objects.all()
gaugeconfigs.filter(short_tag="a15m310L", config=300)

<DataFrameQuerySet [<Nf211: Nf211[GaugeConfig](tag=l2448f211b580m013m065m838, short_tag=a15m310L, stream=a, config=300, nx=24, ny=24, nz=24, nt=48, mpi=310)>]>

In [ ]:


fermionaction_linksmear, created = linksmear_WilsonFlow.objects.get_or_create(
	flowtime=, # Flow time in lattice units
	flowstep=, # Number of diffusion steps
	tag=, # (Optional) User defined tag for easy searches
)

fermionaction, created = fermionaction_MobiusDW.objects.get_or_create(
	quark_mass=, # Input quark mass
	quark_tag=, # Type of quark
	l5=, # Length of 5th dimension
	m5=, # 5th dimensional mass
	b5=, # Mobius kernel parameter [a5 = b5 - c5, alpha5 * a5 =…
	c5=, # Mobius kernal perameter
	linksmear=fermionaction_linksmear, # Foreign Key pointing to additional gauge `linksmear` outside of Monte Carlo.
	tag=, # (Optional) User defined tag for easy searches
)

sourcesmear, created = quarksmear_GaugeCovariantGaussian.objects.get_or_create(
	radius=, # Smearing radius in lattice units
	step=, # Number of smearing steps
	tag=, # (Optional) User defined tag for easy searches
	description=, # (Optional) Description of the quark smearing operator
)

sinksmear, created = quarksmear_Point.objects.get_or_create(
	tag=, # (Optional) User defined tag for easy searches
	description=, # (Optional) Description of the quark smearing operator
)

propagator_onetoall, created = propagator_OneToAll.objects.get_or_create(
	gaugeconfig=gaugeconfig, # Foreign Key pointing to specific \texttt{gaugeconfig} inverted on
	fermionaction=fermionaction, # Foreign Key pointing to valence lattice `fermionaction`. This is the valence action.
	origin_x=, # x-coordinate origin location of the propagator
	origin_y=, # y-coordinate origin location of the propagator
	origin_z=, # z-coordinate origin location of the propagator
	origin_t=, # t-coordinate origin location of the propagator
	sourcesmear=sourcesmear, # Foreign Key pointing to source `quarksmear`
	sinksmear=sinksmear, # Foreign Key pointing to sink `quarksmear`
	tag=, # (Optional) User defined tag for easy searches
)